Necessary Imports

In [2]:
import os
import sys
import pandas as pd
import numpy as np
from PIL import Image

import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score

sys.path.append(os.path.abspath(os.path.join('..', '..', 'models', 'cnn')))
from cnn import CNN
from multilabel_cnn import MultiLabelCNN


Implement a function load mnist data() that extracts images from the
dataset folders and organizes them into separate lists for images and labels
corresponding to the train, validation, and test splits. Ensure that the
images are loaded from their respective folders without any overlap or
mixing.

In [3]:
def one_hot_encode_digit(digit):
    """Creates an 11-dimensional one-hot vector for a single digit or blank."""
    one_hot = np.zeros(11)
    if digit == '*':
        one_hot[10] = 1 
    else:
        one_hot[int(digit)] = 1
    return one_hot

def one_hot_encode_label(label):
    """Encodes the label (up to three digits) as a 33-dimensional vector."""
    label_str = str(label).ljust(3, '*')
    one_hot_encoded = np.concatenate([one_hot_encode_digit(digit) for digit in label_str])
    return one_hot_encoded


def load_mnist_data(base_path='../../data/external/double_mnist', use_label_count=False):
    images = {
        'train': [],
        'val': [],
        'test': []
    }
    labels = {
        'train': [],
        'val': [],
        'test': []
    }

    splits = ['train', 'val', 'test']
    for split in splits:
        split_path = os.path.join(base_path, split)
        for label in os.listdir(split_path):
            label_path = os.path.join(split_path, label)
            if os.path.isdir(label_path):
                for img_name in os.listdir(label_path):
                    img_path = os.path.join(label_path, img_name)
                    img = Image.open(img_path).convert('L')
                    img_array = np.array(img)
                    images[split].append(img_array)

                    if use_label_count:
                        if label == "0":
                            label_count = 0
                        else:
                            label_count = len(label)
                        labels[split].append(label_count)
                    else:
                        labels[split].append(one_hot_encode_label(label))

    return images, labels

images, labels = load_mnist_data(use_label_count=True)


Create a class called MultiMNISTDataset that will be used to create
dataloaders for training and evaluation purposes.

In [4]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

class MultiMNISTDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img = self.images[idx]
        label = self.labels[idx]
        
        img = torch.tensor(img, dtype=torch.float32) / 255.0  

        if self.transform:
            img = self.transform(img)

        return img, label

transform = transforms.Compose([
    transforms.ToPILImage(),         
    transforms.Resize((28, 28)),     
    transforms.ToTensor(),         
])

train_dataset = MultiMNISTDataset(images['train'], labels['train'], transform=transform)
val_dataset = MultiMNISTDataset(images['val'], labels['val'], transform=transform)
test_dataset = MultiMNISTDataset(images['test'], labels['test'], transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


Implement the CNN Class

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = CNN(task='regression', num_classes=4, num_conv_layers=1,dropout_rate=0.5, optimizer_choice='adam', device=device).to(device)
model.fit(train_loader, val_loader, epochs=5, lr=0.001)

./figures/cnn_loss_plots/plot.png


Training Epoch 1/5:  32%|███▏      | 125/394 [00:04<00:08, 31.03it/s]


KeyboardInterrupt: 

In [6]:
import pandas as pd
import torch
from sklearn.metrics import accuracy_score

def calculate_accuracy(model, test_loader, output_file='predictions.csv'):
    model.eval()
    predictions = model.predict(test_loader)
    
    true_labels = []
    for _, labels in test_loader:
        true_labels.append(labels)
    true_labels = torch.cat(true_labels).cpu()
    predictions = torch.round(predictions).cpu()
    
    # Create a DataFrame to display true labels, predictions, and correctness
    results_df = pd.DataFrame({
        'True Labels': true_labels.numpy(),
        'Predictions': predictions.numpy(),
        'Correctness': ['✓' if true == pred else 'x' for true, pred in zip(true_labels.numpy(), predictions.numpy())]
    })
    
    print(results_df.head())
    results_df.to_csv(output_file, index=False)
    print(f"Results saved to {output_file}")
    
    accuracy = accuracy_score(true_labels, predictions)
    print(f"Test Accuracy: {accuracy:.4f}")
    return accuracy

accuracy = calculate_accuracy(model, test_loader)


   True Labels  Predictions Correctness
0            3          3.0           ✓
1            3          3.0           ✓
2            3          3.0           ✓
3            3          2.0           x
4            3          2.0           x
Results saved to predictions.csv
Test Accuracy: 0.7967


In [15]:
import os
import json
import torch.nn as nn
import torch

hyperparam_combinations = [
    {'lr': 0.001, 'dropout_rate': 0.3, 'num_conv_layers': 2, 'optimizer_choice': 'adam'},
    {'lr': 0.001, 'dropout_rate': 0.5, 'num_conv_layers': 3, 'optimizer_choice': 'adam'},
    {'lr': 0.0005, 'dropout_rate': 0.5, 'num_conv_layers': 2, 'optimizer_choice': 'sgd'},
    {'lr': 0.0005, 'dropout_rate': 0.3, 'num_conv_layers': 3, 'optimizer_choice': 'sgd'},
    {'lr': 0.0001, 'dropout_rate': 0.5, 'num_conv_layers': 4, 'optimizer_choice': 'adam'}
]

plot_save_dir = "./figures/cnn_loss_plots"
os.makedirs(plot_save_dir, exist_ok=True)

best_classification_model, best_classification_loss, best_classification_combination = None, float('inf'), None
best_regression_model, best_regression_loss, best_regression_combination = None, float('inf'), None

for task in ['classification', 'regression']:
    for idx, params in enumerate(hyperparam_combinations, 1):
        loss_figure_save_path = os.path.join(
            plot_save_dir,
            f"{task}_loss_combination_{idx}_lr_{params['lr']}_dropout_{params['dropout_rate']}_layers_{params['num_conv_layers']}_opt_{params['optimizer_choice']}.png"
        )
        model = CNN(
            task=task,
            num_classes=4 if task == 'classification' else 1,
            num_conv_layers=params['num_conv_layers'],
            dropout_rate=params['dropout_rate'],
            optimizer_choice=params['optimizer_choice'],
            device='cuda' if torch.cuda.is_available() else 'cpu',
            loss_figure_save_path = loss_figure_save_path
        )

        model.fit(train_loader, val_loader, epochs=2, lr=params['lr'])
        
        criterion = nn.CrossEntropyLoss() if task == 'classification' else nn.MSELoss()
        val_loss = model.evaluate(val_loader, criterion=criterion)
        
        if task == 'classification':
            if val_loss < best_classification_loss:
                best_classification_model = model
                best_classification_loss = val_loss
                best_classification_combination = params
        else:  # task == 'regression'
            if val_loss < best_regression_loss:
                best_regression_model = model
                best_regression_loss = val_loss
                best_regression_combination = params

best_params = {
    "best_classification_combination": best_classification_combination,
    "best_regression_combination": best_regression_combination
}

params_file_path = os.path.join(plot_save_dir, "best_model_params.json")
with open(params_file_path, 'w') as json_file:
    json.dump(best_params, json_file, indent=4)

print("Best Classification Model Combination:", best_classification_combination)
print("Best Regression Model Combination:", best_regression_combination)
print(f"Best parameters saved to {params_file_path}")

./figures/cnn_loss_plots/classification_loss_combination_1_lr_0.001_dropout_0.3_layers_2_opt_adam.png


Training Epoch 1/2: 100%|██████████| 394/394 [00:14<00:00, 27.04it/s]


Epoch [1/2], Training Loss: 0.2117, Validation Loss: 0.0053


Training Epoch 2/2: 100%|██████████| 394/394 [00:14<00:00, 26.96it/s]


Epoch [2/2], Training Loss: 0.0083, Validation Loss: 0.0032
./figures/cnn_loss_plots/classification_loss_combination_2_lr_0.001_dropout_0.5_layers_3_opt_adam.png


...

Epoch [2/2], Training Loss: 0.2516, Validation Loss: 0.1488
./figures/cnn_loss_plots/regression_loss_combination_5_lr_0.0001_dropout_0.5_layers_4_opt_adam.png


Training Epoch 1/2: 100%|██████████| 394/394 [00:12<00:00, 31.58it/s]


Epoch [1/2], Training Loss: 0.6441, Validation Loss: 0.1403


Training Epoch 2/2: 100%|██████████| 394/394 [00:12<00:00, 30.79it/s]


Epoch [2/2], Training Loss: 0.1625, Validation Loss: 0.0365
Best Classification Model Combination: {'lr': 0.001, 'dropout_rate': 0.3, 'num_conv_layers': 2, 'optimizer_choice': 'adam'}
Best Regression Model Combination: {'lr': 0.001, 'dropout_rate': 0.3, 'num_conv_layers': 2, 'optimizer_choice': 'adam'}
Best parameters saved to ./figures/cnn_loss_plots/best_model_params.json


In [ ]:
def read_best_params(file_path):
    with open(file_path, 'r') as json_file:
        params = json.load(json_file)
    return params

loaded_params = read_best_params(params_file_path)
print("Loaded Parameters:", loaded_params)


In [ ]:
images, labels = load_mnist_data(use_label_count=False)

train_dataset = MultiMNISTDataset(images['train'], labels['train'], transform=transform)
val_dataset = MultiMNISTDataset(images['val'], labels['val'], transform=transform)
test_dataset = MultiMNISTDataset(images['test'], labels['test'], transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MultiLabelCNN(device=device).to(device)
model.fit(train_loader, val_loader, epochs=1, lr=0.001)

Training Epoch 1/1: 100%|██████████| 394/394 [00:07<00:00, 49.53it/s]


Epoch [1/1], Training Loss: 4.9962
Epoch [1/1], Validation Loss: 5.3681


In [ ]:
def exact_match_accuracy(preds, labels):
    """Calculates the exact match accuracy between predictions and labels."""
    return (preds == labels).all(dim=1).float().mean().item()

def hamming_accuracy(preds, labels):
    """Calculates the Hamming accuracy for multi-label predictions."""
    return (preds == labels).float().mean().item()


def evaluate_model(model, test_loader):
    preds = model.predict(test_loader)
    labels = torch.cat([labels for _, labels in test_loader])
    print(preds[12], labels[12])

    exact_acc = exact_match_accuracy(preds, labels)
    hamming_acc = hamming_accuracy(preds, labels)
    print(f"Exact Match Accuracy: {exact_acc * 100:.2f}%")
    print(f"Hamming Accuracy: {hamming_acc * 100:.2f}%")

evaluate_model(model, test_loader)


tensor([0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.]) tensor([0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       dtype=torch.float64)
Exact Match Accuracy: 2.43%
Hamming Accuracy: 89.70%
